# Verify behavior cloning

Visually verify a behavior cloning based robot controller.

__TODO as of June 1, 2025__

* Load the controller trained by the Train_BC notebook and the corresponding sp
* Choose a demonstration, to which the SP/BC pair will be applied to the images
* Visualize the path taken by the demonstration and what would be (autoregressively) recommended by the controller

In [ ]:
import sys
sys.path.append("..")

from exp_run_config import Config
Config.PROJECTNAME = "BerryPicker"

import pathlib
from tqdm import tqdm
import numpy as np
import pprint
#import matplotlib.pyplot as plt

import torch
import torch.nn as nn
#import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)
from sensorprocessing.sp_helper import get_transform_to_sp
from sensorprocessing.sp_factory import create_sp
from demonstration.demonstration import Demonstration

from bc_LSTM import LSTMXYPredictor, LSTMResidualController
from robot.al5d_position_controller import RobotPosition

from tensorboardX import SummaryWriter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
experiment = "behavior_cloning"
# run = "bc_mlp_00"
run = "lstm_00"

exp = Config().get_experiment(experiment, run)
pprint.pprint(exp)

# Create the sp object described in the experiment
spexp = Config().get_experiment(exp["sp_experiment"], exp["sp_run"])
sp = create_sp(spexp, device)

In [ ]:
if exp["controller"] != "LSTMXYPredictor":
    raise Exception(f"The controller specified {exp['controller']} is not LSTMXYPredictor")

latent_size = spexp["latent_size"]
output_size = exp["control_size"]  # degrees of freedom in the robot
num_layers = exp["controller_num_layers"]
hidden_size = exp["controller_hidden_size"] 

# Instantiate model, loss function, and optimizer
model = LSTMXYPredictor(latent_size=latent_size, hidden_size=hidden_size, output_size = output_size, num_layers=num_layers)

controller_path = pathlib.Path(exp.data_dir(), exp["controller_file"])
model.load_state_dict(torch.load(controller_path))